# Set up the connection

In [1]:
import paramiko
import os
import sys
import select
import time 

In [15]:
class SSHShell:
    def __init__(self, host, user, psw, port):
        self.ssh = paramiko.SSHClient()
        self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.ssh.connect(host, username=user, password=psw, port=port, timeout=2)
        self.refresh()

    def __del__(self):
        self.ssh.close()
    
    def refresh(self):
        self.channel = self.ssh.invoke_shell()
        self.stdin = self.channel.makefile('wb')
        self.stdout = self.channel.makefile('r')    

    def execute(self, cmd):
        cmd = cmd.strip('\n')
        self.stdin.write(cmd + '\n')
        self.stdin.flush()
        time.sleep(0.5)
        while self.stdout.channel.recv_ready():
            rl, wl, xl = select.select([ self.stdout.channel ], [ ], [ ])
            if len(rl) > 0:
                tmp = self.stdout.channel.recv(1024)
                output = tmp.decode()
                print(output)

In [16]:
CLI = SSHShell("10.120.217.249", "brocmyp", "planbyang", 2222)

In [4]:
CLI.execute('screen-length 100')

Welcome
Last login time: First Login

brocmyp connected from 10.120.217.52 using ssh on eric-cm-yang-provider-59df9f448d-gtthw
brocmyp@eric-cm-yang-provider-59df9f448d-gtthw#screen-length 100
brocmyp@eric-cm-yang-provider-59df9f448d-gtthw#


## BRM

In [5]:
CLI.execute('show running-config brm')

show running-config brm
brm backup-manager DEFAULT-eric-data-document-database-pg
 housekeeping max-stored-manual-backups 1
 housekeeping auto-delete  enabled
 scheduler scheduled-backup-name SCHEDULED_BACKUP
 scheduler auto-export disabled
 scheduler admin-state unlocked
!
brm backup-manager DEFAULT
 housekeeping max-stored-manual-backups 1
 housekeeping auto-delete  enabled
 scheduler scheduled-backup-name SCHEDULED_BACKUP
 scheduler auto-export disabled
 scheduler admin-state unlocked
!
brocmyp@eric-cm-yang-provider-59df9f448d-gtthw#


In [7]:
BRM = 'DEFAULT'

In [8]:
VBRM = "DEFAULT-eric-data-document-database-pg"

## Backup

### List backups

In [12]:
CLI.execute(f'show brm backup-manager {BRM} backup')

return-value 39402
brocmyp@eric-cm-yang-provider-59df9f448d-gtthw#show brm backup-manager DEFAULT backup
backup cmypBackup01
 backup-name   cmypBackup01
 creation-time 2022-02-23T11:23:19.866118+00:00
 status        backup-complete
 creation-type manual
 sw-version "APR 201 34" 7.1.0-43
  product-name    "Document Database PG"
  production-date 2022-02-23T11:23:19.938858+00:00
  description     "eric-data-document-database-pg Agent for backup and restore."
  type            Database
 sw-version "APR XXX XX/X" R1A
  product-name    "Test Application"
  production-date 2019-05-17T14:12:44+00:00
  description     "Test Application Drop 15"
  type            Predelivery
brocmyp@eric-cm-yang-provider-59df9f448d-gtthw#


### Detail a backup

In [266]:
CLI.execute(f'show brm backup-manager {BRM} backup {backupName1}')

show brm backup-manager DEFAULT backup cmypBackup01
backup cmypBackup01
 backup-name   cmypBackup01
 creation-time 2022-02-18T16:18:39.459365+00:00
 status        backup-complete
 creation-type manual
 progress-report 13398
  action-name                EXPORT
  additional-info            [ "Exported Backup: cmypBackup01-2022-02-18T16:18:39.459365Z.tar.gz" ]
  progress-percentage        100
  result                     success
  state                      finished
  time-action-started        2022-02-18T16:33:57.2157+00:00
  time-action-completed      2022-02-18T16:33:58.835425+00:00
  time-of-last-status-update 2022-02-18T16:33:58.835425+00:00
 sw-version "APR 201 34" 7.1.0-19
  product-name    "Document Database PG"
  production-date 2022-02-18T16:31:18.122509+00:00
  description     "eric-data-document-database-pg Agent for backup and restore."
  type            Database
 sw-version "APR XXX XX/X" R1A
  product-name    "Test Application"
  production-date 2019-05-17T14:12:44+00:00
  

### Get an attribute from a Backup

In [212]:
CLI.execute(f'show brm backup-manager {BRM} backup {backupName1} sw-version')

show brm backup-manager DEFAULT backup cmypBackup01 sw-version

sw-version "APR 201 34" 7.1.0-19
 product-name    "Document Database PG"
 production-date 2022-02-18T16:18:39.579782+00:00
 description     "eric-data-document-database-pg Agent for backup and restore."
 type            Database
sw-version "APR XXX XX/X" R1A
 product-name    "Test Application"
 production-date 2019-05-17T14:12:44+00:00
 description     "Test Application Drop 15"
 type            Predelivery
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


### create a backup

In [9]:
backupName1 = 'cmypBackup01'

In [10]:
backupName2 = 'cmypBackup02'

In [11]:
CLI.execute(f'brm backup-manager {BRM} create-backup name {backupName1}')

brm backup-manager DEFAULT create-backup name cmypBackup01



### Restore a backup

In [215]:
CLI.execute(f'brm backup-manager {BRM} backup {backupName1} restore')

brm backup-manager DEFAULT backup cmypBackup01 restore


return-value 10326
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#
Restore is about to start, sessions may be terminated soon!



In [18]:
CLI.execute(f'brm backup-manager DEFAULT-agent-three backup {backupName1} restore')

brm backup-manager DEFAULT-agent-three backup cmypBackup01 restore
return-value 45387
brocmyp@eric-cm-yang-provider-59df9f448d-gtthw#


In [156]:
CLI.execute(f'brm backup-manager {VBRM} backup {backupName1} restore')

Welcome
Last login time: First Login

brocmyp connected from 10.120.217.52 using ssh on eric-cm-yang-provider-5bbb75c878-jwdth
brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth#
brm backup-manager DEFAULT-eric-data-document-database-pg backup cmypBackup01 restore


return-value 14172
brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth#
Restore is about to start, sessions may be terminated soon!



### Export a backup

In [249]:
CLI.refresh()

In [218]:
uri = 'sftp://brsftp@bro-test-sftp:9022/bro_test'
pwd = "planbsftp"

In [220]:
CLI.execute(f'brm backup-manager {BRM} backup {backupName1} export uri {uri} password {pwd}')

brm backup-manager DEFAULT backup cmypBackup01 export uri sftp://brsftp@bro-test-sftp:9022/bro_test password planbsftp

return-value 13398
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


### Delete a backup

In [144]:
CLI.execute(f'brm backup-manager {VBRM} delete-backup name {backupName2}')

brm backup-manager DEFAULT-eric-data-document-database-pg delete-backup name cmypBackup02

return-value 3819
brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth#


### Import a backup

In [150]:
CLI.execute(f"brm backup-manager {BRM} import-backup uri {uri}/{BRM}/{backupName1} password {pwd}")

brm backup-manager DEFAULT import-backup uri sftp://brsftp@bro-test-sftp:9022/bro_test/DEFAULT/cmypBackup01 password planbsftp


return-value 40760
brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth#


## Housekeeping

### Get BRM's housekeeping

In [248]:
CLI.execute(f'show running-config brm backup-manager {BRM} housekeeping')

OSError: Socket is closed

### Update housekeeping

In [160]:
CLI.execute(f'config')

config

Entering configuration mode private
brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth(config)#


In [161]:
CLI.execute(f'brm backup-manager {BRM} housekeeping max-stored-manual-backups 20')

brm backup-manager DEFAULT housekeeping max-stored-manual-backups 20

brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth(config)#


In [162]:
CLI.execute(f'brm backup-manager {BRM} housekeeping auto-delete disabled')

brm backup-manager DEFAULT housekeeping auto-delete disabled

brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth(config)#


In [174]:
CLI.execute(f'commit')

commit

Aborted:'eypvalid:yang-provider-validation': Failed to convert the value

brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth(config)#


In [177]:
CLI.execute(f'exit')

exit


Uncommitted changes found, commit them? [yes/no/CANCEL] 


In [178]:
CLI.execute('no')

no

brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth#


## Scheduler

### show runing config

In [243]:
CLI.execute(f'show running-config brm backup-manager {BRM} scheduler')

show running-config brm backup-manager DEFAULT scheduler
brm backup-manager DEFAULT
 scheduler scheduled-backup-name BACKUP
 scheduler auto-export enabled
 scheduler auto-export-uri sftp://brsftp@bro-test-sftp:9022/bro_test
 scheduler auto-export-password ********
 scheduler admin-state locked
!
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


### Update runing config

In [246]:
CLI.execute(f"""config 
            brm backup-manager DEFAULT scheduler admin-state locked
    brm backup-manager DEFAULT scheduler scheduled-backup-name BACKUP
    brm backup-manager DEFAULT scheduler auto-export enabled
    brm backup-manager DEFAULT scheduler auto-export-uri {uri}
    brm backup-manager DEFAULT scheduler auto-export-password {pwd}
    commit
    exit
    """)

config 
-------------------------------------------------------^
syntax error: unknown command
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#            brm backup-manager DEFAULT scheduler admin-state locked



In [193]:
CLI.execute(f"config") 

config

Entering configuration mode private
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#


In [194]:
CLI.execute(f"brm backup-manager DEFAULT scheduler auto-export-uri {uri}") 

brm backup-manager DEFAULT scheduler auto-export-uri sftp://brsftp@bro-test-sftp:9022/bro_test

brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#


In [195]:
CLI.execute(f"brm backup-manager DEFAULT scheduler auto-export-password {pwd}") 

brm backup-manager DEFAULT scheduler auto-export-password planbsftp

brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#


In [196]:
CLI.execute(f'commit')

commit


Commit complete.
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#


### show BRM's scheduler

In [200]:
CLI.execute(f'show brm backup-manager {BRM} scheduler')

show brm backup-manager DEFAULT scheduler

% No entries found.
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


### show scheduler progress-report

In [214]:
CLI.execute(f'show brm backup-manager {BRM} scheduler progress-report 123')

show brm backup-manager DEFAULT scheduler progress-report 123

---------------------------------------------------------------------------------------------------------^
syntax error: unknown argument
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


## Action

### List Actions

In [213]:
CLI.execute(f'show brm backup-manager {BRM} action')

show brm backup-manager DEFAULT action


-------------------------------------------------------------------------------^
syntax error: unknown argument
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


### Progress report

### show restore/export progress-report

In [221]:
CLI.execute(f'show brm backup-manager {BRM} backup {backupName1} progress-report 13398')

show brm backup-manager DEFAULT backup cmypBackup01 progress-report 13398


progress-report 13398
 action-name                EXPORT
 additional-info            [ "Exported Backup: cmypBackup01-2022-02-18T16:18:39.459365Z.tar.gz" ]
 progress-percentage        100
 result                     success
 state                      finished
 time-action-started        2022-02-18T16:33:57.2157+00:00
 time-action-completed      2022-02-18T16:33:58.835425+00:00
 time-of-last-status-update 2022-02-18T16:33:58.835425+00:00
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


### show create import delete progress report

In [245]:
CLI.execute(f'show brm backup-manager {BRM} progress-report 63598')

Commit complete.
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#    
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#show brm backup-manager DEFAULT progress-report 63598
------------------------------------------------------------^
syntax error: expecting 
  configuration      - Show a parameter
  full-configuration - Show a parameter
  history            - Display CLI command history
  parser             - Display parser information
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#


# other command

In [216]:
CLI.refresh()

In [199]:
CLI.execute(f'commit')

Welcome
Last login time: First Login

brocmyp connected from 10.120.217.52 using ssh on eric-cm-yang-provider-5bbb75c878-629tm
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#
commit

-----------------------------------------------^
syntax error: expecting 
  autowizard           - Automatically query for mandatory elements
  change-password      - Change the password of current user
  clear                - Clear parameter
  complete-on-space    - Enable/disable completion on space
  config               - Manipulate software configuration information
  describe             - Display transparent command  information
  display-level        - Configure show command display level
  exit                 - Exit the management session
  help                 - Provide help information
  history              - Configure history size
  idle-timeout         - Configure idle timeout
  ignore-leading-space - Ignore leading whitespace (true/false)
  leaf-prompting       - Automatically query for le

In [247]:
CLI.execute(f'exit')

brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#    brm backup-manager DEFAULT scheduler scheduled-backup-name BACKUP
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#    brm backup-manager DEFAULT scheduler auto-export enabled
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#    brm backup-manager DEFAULT scheduler auto-export-uri sftp://brsftp@bro-test-sftp:9022/bro_test
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#    brm backup-manager DEFAULT scheduler auto-export-password planbsftp
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#    commit
Commit complete.
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#    exit
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#    
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#exit

